** Vector Space Model**

I have done this project on Google Colaboratory. So you can just click this shareable link (https://colab.research.google.com/drive/1Oio0HMNVipLcO_vTrNiaQxB45ef0TgqJ) and run.

OR you can download the notebook and run on local machine or whatever you want.

I will upadate the notebook with more optimized code. This time it takes a lot of time and memory.

LINK FOR DATASET: https://drive.google.com/open?id=10pTRD0Yg-QR3v_j7Rr-RAT2HiN8Ro6Pq

**Objective:** 

*Objective of this project is to implement the Vector Space Model (VSM). Dataset contains a collection of about 22,000 text files.*

*Assigned task was to implement a VSM and given a query, program should return top 5 documents related to the query.*

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from zipfile import ZipFile
file_name = '/content/drive/My Drive/ACL Text.zip'

with ZipFile(file_name, 'r',) as zip:
  zip.extractall()
  print('Done!!')

Done!!


**Bellow cell imports all the necessary libraries**

In [0]:
import glob
import nltk
nltk.download('popular');
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
from collections import Counter
import numpy as np
from collections import OrderedDict

**give_path: **function** **

In below cell, give_path() takes path of the folder containing all the documents.
give_path() then read all the documents and store content of each document (in string format) as 'value' of dictionary and it's file name as 'key' in the dictionary.
This function the returns the dictionary containing all documents.

In [0]:
def give_path(fld_path):                             #give path of the folder containing all documents
    dic = {}
    file_names = glob.glob(fld_path)
    files_150 = file_names[0:10]
    for file in files_150:
        name = file.split('/')[-1]
        with open(file, 'r', errors='ignore') as f:
            data = f.read()
        dic[name] = data
    return dic

**wordList_removePuncs: **function** **

In the below cell, two functions wordList() and removePuncs() functions have been combined as wordList_removePuncs.
This function is passed with dictionary of all documents (returned by give_path()). Function then remove the stop words and punctuations and returns a list of all the words in our collection of documents. This list contains the redundant words.

In [0]:
def wordList_removePuncs(doc_dict):
    stop = stopwords.words('english') + list(string.punctuation) + ['\n']
    wordList = []
    for doc in doc_dict.values():
        for word in word_tokenize(doc.lower().strip()): 
            if not word in stop:
                wordList.append(word)
    return wordList

**termFrequencyInDoc: **function** **

After the above function returns a list of all words in the collection of documents, vocabulary of words is made (I'm making vocabulary in 'main'). This vocabulary and collection of documents in dictionary form (returned by give_path) is passed to below function (termFrequencyInDoc) to find the frequency of words in each documet. This function returns a dictionary in which 'key' is the document name and its 'value' is another dicionary, containing a word as 'key' and its frequency as 'value'. It's actually a dictionary within a dictionary.

In [0]:
def termFrequencyInDoc(vocab, doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id] = {}
    
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_docs[doc_id][word] = doc.count(word)
    return tf_docs

**wordDocFre: **function** **

To count the document frequency, vocabulary of words and dictionary of collection of documents if passed to wordDocFre which returns a dictionary containing a word as 'key' and its document frequency as 'value' of dictionary.

In [0]:
def wordDocFre(vocab, doc_dict):
    df = {}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
#             if word in doc.lower().split():
            if word in word_tokenize(doc.lower().strip()):
                frq = frq + 1
        df[word] = frq
    return df

**inverseDocFre: **function** **

inverseDocFre calculates the inverse document frequency. It takes vocabulary of words, dictionary of document frequency (returned by wordDocFre) and number of documents in our collection. It returns a dictionary containing inverse DF in which a word is key and its idf score is its value.

In [0]:
def inverseDocFre(vocab,doc_fre,length):
    idf= {} 
    for word in vocab:     
        idf[word] = np.log2((length+1) / doc_fre[word])
    return idf

**tfidf: **function** **

Function tfidf takes 4 arguments:
1. vocabular of words 
2. term frequencies: which is passed in form of a dictionary
3. Inverse DF: passed in form of a dictionary
4. Collection of all docs passed in form of a dictionary

It returns a dictionary, which is again a dictionary within a dictionary like in case of TF, but it contains score values.

In [0]:
def tfidf(vocab,tf,idf_scr,doc_dict):
    tf_idf_scr = {}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id] = {}
    for word in vocab:
        for doc_id,doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word] * idf_scr[word]
    return tf_idf_scr

**vectorSpaceModel: **function** **

To find the relevant documents related to query, pass the query to function alonwith collection of documents (dictionary) and tf-idf scores (dictionary returned by tfidf). Function returns the top 5 documents from a collection of about 22k documents.

In [0]:
def vectorSpaceModel(query, doc_dict,tfidf_scr):
    query_vocab = []
    for word in query.split():
        if word not in query_vocab:
            query_vocab.append(word)

    query_wc = {}
    for word in query_vocab:
        query_wc[word] = query.lower().split().count(word)
    
    relevance_scores = {}
    for doc_id in doc_dict.keys():
        score = 0
        for word in query_vocab:
            score += query_wc[word] * tfidf_scr[doc_id][word]
        relevance_scores[doc_id] = score
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    top_5 = {k: sorted_value[k] for k in list(sorted_value)[:5]}
    return top_5

**Here is the main function call which calls all the functions above and gives the results.**

In [0]:
if __name__  == "__main__":
   path = '/content/ACL txt/*.txt'
   docs = give_path(path)                        #returns a dictionary of all docs
   M = len(docs)                                 #number of files in dataset
   w_List = wordList_removePuncs(docs)           #returns a list of tokenized words
   vocab = list(set(w_List))                     #returns a list of unique words
   tf_dict = termFrequencyInDoc(vocab, docs)     #returns term frequency
   df_dict = wordDocFre(vocab, docs)             #returns document frequencies
   idf_dict = inverseDocFre(vocab,df_dict,M)     #returns idf scores
   tf_idf = tfidf(vocab,tf_dict,idf_dict,docs)   #returns tf-idf socres
   
   query1 = 'Text Mining'
   query2 = 'LDA'
   query3 = 'topic modeling'
   query4 = 'Natural language Processing'
   query5 = 'generative models'
   top1 = vectorSpaceModel(query1, docs,tf_idf)    #returns top 5 documents using VSM
   top2 = vectorSpaceModel(query2, docs,tf_idf)    #returns top 5 documents using VSM
   top3 = vectorSpaceModel(query3, docs,tf_idf)    #returns top 5 documents using VSM
   top4 = vectorSpaceModel(query4, docs,tf_idf)    #returns top 5 documents using VSM
   top5 = vectorSpaceModel(query5, docs,tf_idf)    #returns top 5 documents using VSM
   print('Top 5 Documents for Query 1: \n', top1)
   print('\n')
   print('Top 5 Documents for Query 2: \n', top2)
   print('\n')
   print('Top 5 Documents for Query 3: \n', top3)
   print('\n')
   print('Top 5 Documents for Query 4: \n', top4)
   print('\n')
   print('Top 5 Documents for Query 5: \n', top5)